In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib as plt
from itertools import chain
import ast

In [2]:
df = pd.read_csv("full_set.csv")
df.shape

(588793, 31)

In [3]:
TARGET_CLASS = "CHM2210"

# converts strings to arrays
df['Classes'] = df['Classes'].apply(ast.literal_eval)
df['Semester Grades'] = df['Semester Grades'].apply(ast.literal_eval)
df['Semester Points'] = df['Semester Points'].apply(ast.literal_eval)
df['CRN'] = df['CRN'].apply(ast.literal_eval)

# find all people who took class
Pidms_with_TARGET_CLASS = df[df['Classes'].apply(lambda x: TARGET_CLASS in x)]['Pidm'].unique()
df = df[df['Pidm'].isin(Pidms_with_TARGET_CLASS)]
df = df[['Pidm', 'Semester', 'HS GPA', 'Converted_SAT', 'Semester Points', 'Semester Grades', 'CRN', 'Classes']]
df.head(4)

# Find the first semester with TARGET_CLASS for each student
def find_first_semester(student_df):
    chm2210_row = student_df[student_df['Classes'].apply(lambda x: TARGET_CLASS in x)]
    if not chm2210_row.empty:
        return chm2210_row['Semester'].min()
    return None

first_semester = df.groupby('Pidm').apply(lambda x: find_first_semester(x)).rename('Target_Semester')
df = df.merge(first_semester, on='Pidm')

# filter all semesters after student took TARGET_CLASS
filtered_df = df[df['Semester'] <= df['Target_Semester']]

# find grades/points for TARGET_CLASS and output it to a new column
def find_class_grades(student_df):
    for _, row in student_df.iterrows():
        if TARGET_CLASS in row['Classes']:
            index = row['Classes'].index(TARGET_CLASS)
            return row['Semester Points'][index], row['Semester Grades'][index]
    return None, None

class_grades = filtered_df.groupby('Pidm').apply(lambda x: find_class_grades(x)).apply(pd.Series)
class_grades.columns = ['Target_Points', 'Target_Grade']

final_df = filtered_df.merge(class_grades, on='Pidm')

# filter out these grades
final_df = final_df[~final_df['Target_Grade'].isin(['WE', 'IF', 'W', 'WC'])]

display(df[df['Pidm'] ==  134328])
final_df = final_df[final_df['Semester'] < final_df['Target_Semester']]
display(final_df[final_df['Pidm'] ==  134328])
groupped_df = final_df.groupby('Pidm').agg({
    "HS GPA": 'first',
    'Converted_SAT': 'first',
    'Semester Grades': lambda x: sum(x, []),
    'Semester Points': lambda x: sum(x, []),
    'Classes': lambda x: sum(x, []),
    'CRN': lambda x: sum(x, []),
    'Target_Grade': 'first',
    'Target_Points': 'first',
}).reset_index()

all_classes = sorted(set(chain.from_iterable(groupped_df['Classes'])))

def create_one_hot(classes, points, all_classes):
    one_hot_vector = [-1] * len(all_classes)
    for class_name, point in zip(classes, points):
        if class_name in all_classes:
            one_hot_vector[all_classes.index(class_name)] = point
    return one_hot_vector

groupped_df['One_Hot_Classes'] = groupped_df.apply(
    lambda row: create_one_hot(row['Classes'], row['Semester Points'], all_classes), axis=1
)

one_hot_df = pd.DataFrame(groupped_df['One_Hot_Classes'].tolist(), columns=all_classes, index=groupped_df['Pidm'])

from sklearn.model_selection import train_test_split

train, testing_data = train_test_split(one_hot_df, test_size=0.2, random_state=50)
dev, test = train_test_split(testing_data, test_size=0.5, random_state=50)

train_set = one_hot_df[one_hot_df.index.isin(train.index)]
dev_set = one_hot_df[one_hot_df.index.isin(dev.index)]
test_set = one_hot_df[one_hot_df.index.isin(test.index)]
columns_to_remove = []
for column in train_set.columns:
    value_counts = train_set[column].value_counts()
    max_count = value_counts.max()
    non_max_count = value_counts.sum() - max_count
    
    if non_max_count <= 0:
        columns_to_remove.append(column)
        
train_set = train_set.drop(columns=columns_to_remove)
dev_set = dev_set.drop(columns=columns_to_remove)
test_set = test_set.drop(columns=columns_to_remove)

print(train_set.shape, dev_set.shape, test_set.shape)

/var/folders/1_/f10h6scn5x19_qnf0xw8fpyr0000gn/T/ipykernel_6499/2545726050.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_semester = df.groupby('Pidm').apply(lambda x: find_first_semester(x)).rename('Target_Semester')
/var/folders/1_/f10h6scn5x19_qnf0xw8fpyr0000gn/T/ipykernel_6499/2545726050.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  class_grades = filtered_df.groupby('Pidm').apply(lam

,Pidm,Semester,HS GPA,Converted_SAT,Semester Points,Semester Grades,CRN,Classes,Target_Semester
0,134328,201601,3.0,1370.0,"[2.0, 3.0]","[C, B]","[11143.0, 23005.0]","[BSC2010L, BSC2010]",201708
1,134328,201701,3.0,1370.0,[4.0],[A],[11691.0],[ENC1101],201708
2,134328,201705,3.0,1370.0,"[3.0, 3.67]","[B, A-]","[52696.0, 52892.0]","[PSY2012, ARC2211]",201708
3,134328,201708,3.0,1370.0,"[1.0, 1.33, 0.0, 2.33]","[D, D+, F, C+]","[86672.0, 87939.0, 88312.0, 91803.0]","[CHM2210, PCB3063, CHM2210L, EGS3720]",201708
4,134328,201808,3.0,1370.0,"[4.0, 0.0]","[A, S]","[80885.0, 89183.0]","[COP2510, EGN3000]",201708
5,134328,201901,3.0,1370.0,"[1.0, 0.0, 0.0]","[D, F, F]","[17655.0, 18741.0, 19361.0]","[COP3514, COT3100, CDA3103]",201708


,Pidm,Semester,HS GPA,Converted_SAT,Semester Points,Semester Grades,CRN,Classes,Target_Semester,Target_Points,Target_Grade
0,134328,201601,3.0,1370.0,"[2.0, 3.0]","[C, B]","[11143.0, 23005.0]","[BSC2010L, BSC2010]",201708,1.0,D
1,134328,201701,3.0,1370.0,[4.0],[A],[11691.0],[ENC1101],201708,1.0,D
2,134328,201705,3.0,1370.0,"[3.0, 3.67]","[B, A-]","[52696.0, 52892.0]","[PSY2012, ARC2211]",201708,1.0,D


(9424, 1433) (1178, 1433) (1179, 1433)


In [4]:
# Create pass/fail column
def map_pass_fail(grade):
    fail_grades = ['F', 'IF', 'W', 'D-', 'F', 'D+', 'D#', 'D+', 'F#', 'D', 'D', 'D-', 'U', 'W', 'F*', 'D*', 'CF', 'I', 'FF', 'Z', 'W*', 'F+', 'F-', 'F#', 'F*', 'D-*', 'IF', 'IF*', 'D+*', 'CIF', 'Z*', 'IU', 'M', 'CI', 'MU', 'U*', 'ID', 'IB', 'IU*', 'IS', 'CW']
    return 0 if grade in fail_grades else 1  # 0 = fail, 1 = pass

#Add columns fro grouped_df
groupped_df_filtered = groupped_df[groupped_df['Pidm'].isin(train_set.index)]
add_columns = ['HS GPA', 'Converted_SAT', 'Target_Grade']
groupped_df_filtered.set_index('Pidm', inplace=True)
train_set = train_set.join(groupped_df_filtered[add_columns], )

groupped_df_filtered = groupped_df[groupped_df['Pidm'].isin(dev_set.index)]
add_columns = ['HS GPA', 'Converted_SAT', 'Target_Grade']
groupped_df_filtered.set_index('Pidm', inplace=True)
dev_set = dev_set.join(groupped_df_filtered[add_columns], )

# Apply the function to create pass/fail column
train_set['pass_fail'] = train_set['Target_Grade'].apply(map_pass_fail)
dev_set['pass_fail'] = dev_set['Target_Grade'].apply(map_pass_fail)

# # Drop irrelevant columns
X = train_set.drop(columns=['Target_Grade', 'pass_fail'])
X_dev = dev_set.drop(columns=['Target_Grade', 'pass_fail'])

# Convert all remaining object columns to numeric
# X = X.apply(pd.to_numeric, errors='coerce')

# Drop rows with missing values
X = X.dropna()
X_dev = X_dev.dropna()

# Extract the target variable (pass/fail)
y = train_set.loc[X.index, 'pass_fail']
y_dev = dev_set.loc[X_dev.index, 'pass_fail']

In [5]:
counts = train_set['pass_fail'].value_counts()
counts.get(1, 1)
print(counts.get(0, 0))
print(counts.get(1, 1))

1122
8302


In [6]:
from sklearn.utils import resample
# Apply undersampling to balance the classes
# Separate the pass and fail instances
pfcounts = train_set['pass_fail'].value_counts()
sample_count = min(counts.get(0, 0), counts.get(1, 1))

pass_class = X[y == 1]
fail_class = X[y == 0]

pass_sample = resample(pass_class, replace=False, n_samples=sample_count, random_state=50)
fail_sample = resample(fail_class, replace=False, n_samples=sample_count, random_state=50)

# Combine the undersampled data
X_undersampled = pd.concat([pass_sample, fail_sample])
y_undersampled = pd.concat([y[pass_sample.index], y[fail_sample.index]])

In [12]:
df1=y_undersampled
df1.shape

(2244,)

In [13]:
print(counts.get(0, 0))
print(counts.get(1, 1))
print(sample_count)
display(pass_sample)
display(fail_sample)

1122
8302
1122


ACG2021  ACG2071  ACG3074  ACG3103  ACG3341  ADV2214  ADV3008  \
Pidm                                                                     
3570131     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3103513     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3102668     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3400610     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3349080     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
...          ...      ...      ...      ...      ...      ...      ...   
3521853     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3632792     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3600128     -1.0     -1.0      4.0     -1.0     -1.0     -1.0     -1.0   
3350916     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3401151     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFA2000  AFA2380  AFA3110  AFA4150  AFA4335  AFA4430  AFA4931  \
Pidm                                                                     
3570131     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3103513     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3102668     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3400610     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3349080     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
...          ...      ...      ...      ...      ...      ...      ...   
3521853     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3632792     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3600128     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3350916     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3401151     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFH3100  AFH3200  AFR1101  AFR1120  AFR2001  AFR2130  AFR2140  \
Pidm                                                                     
3570131     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3103513     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3102668     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3400610     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3349080     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
...          ...      ...      ...      ...      ...      ...      ...   
3521853     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3632792     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3600128     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3350916     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3401151     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFR3220  AFR3231  AFR4201  AFS2250  AMH2010  AMH2020  AMH3140  \
Pidm                                                                     
3570131     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3103513     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3102668     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3400610     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3349080     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
...          ...      ...      ...      ...      ...      ...      ...   
3521853     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3632792     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3600128     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3350916     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3401151     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AMH3160  AMH3170  AMH3270  AMH3403  AMH3421  AMH3423  AMH3533  \
Pidm                                                                     
3570131  

ACG2021  ACG2071  ACG3074  ACG3103  ACG3341  ADV2214  ADV3008  \
Pidm                                                                     
3506043     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3411183     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3084566     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3698977     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3648088     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
...          ...      ...      ...      ...      ...      ...      ...   
3811875     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3191914     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3072611     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3124988     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3109120     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFA2000  AFA2380  AFA3110  AFA4150  AFA4335  AFA4430  AFA4931  \
Pidm                                                                     
3506043     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3411183     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3084566     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3698977     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3648088     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
...          ...      ...      ...      ...      ...      ...      ...   
3811875     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3191914     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3072611     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3124988     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3109120     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFH3100  AFH3200  AFR1101  AFR1120  AFR2001  AFR2130  AFR2140  \
Pidm                                                                     
3506043     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3411183     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3084566     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3698977     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3648088     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
...          ...      ...      ...      ...      ...      ...      ...   
3811875     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3191914     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3072611     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3124988     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3109120     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AFR3220  AFR3231  AFR4201  AFS2250  AMH2010  AMH2020  AMH3140  \
Pidm                                                                     
3506043     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3411183     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3084566     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3698977     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3648088     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
...          ...      ...      ...      ...      ...      ...      ...   
3811875     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3191914     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3072611     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3124988     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   
3109120     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0     -1.0   

         AMH3160  AMH3170  AMH3270  AMH3403  AMH3421  AMH3423  AMH3533  \
Pidm                                                                     
3506043  

In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Training and testing data
X_train = X_undersampled
y_train = y_undersampled
#X_train = X
#y_train = y
#X_test = X_dev
#y_test = y_dev
X_train, X_test, y_train, y_test = train_test_split(X_undersampled, y_undersampled, test_size=0.2, random_state=50)

# Initialize AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(random_state=42)

# Define parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of boosting rounds
    'learning_rate': [0.01, 0.1, 0.5, 1.0]  # Learning rate
}

# Initialize GridSearchCV with the AdaBoost classifier
grid_search = GridSearchCV(estimator=adaboost_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Cross-Validation Accuracy: {best_score:.2f}")

# Train the final model using the best parameters
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Label Model
print("############ AdaBoost Classifier ############")

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")

class_report = classification_report(y_test, y_pred)
print(f"Classification Report:\n{class_report}")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/ana

Best Parameters: {'learning_rate': 0.5, 'n_estimators': 50}
Best Cross-Validation Accuracy: 0.76
############ AdaBoost Classifier ############
Accuracy: 0.76
Confusion Matrix:
[[181  49]
 [ 60 159]]
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.79      0.77       230
           1       0.76      0.73      0.74       219

    accuracy                           0.76       449
   macro avg       0.76      0.76      0.76       449
weighted avg       0.76      0.76      0.76       449

